In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
drive.mount('/content/gdrive')
bengali_news_after_preprocessing = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/Bengali_Text_after_preprocessing.pkl')
from sklearn.externals import joblib
filename = '/content/gdrive/My Drive/Projects/Bengali Text Classification/fastText_Bangla_content_full.sav'
loaded_model = joblib.load(filename)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
import keras.backend as K
import numpy as np
number_of_sample, max_number_of_words, word_vector_size = 40000, 50, 32
temp = bengali_news_after_preprocessing.loc[:number_of_sample-1,:max_number_of_words-1]

Using TensorFlow backend.


In [0]:
temp = temp.replace(['ঘস', 'ফগ', 'ঝবঃ', 'ঋন', 'ঊঘ', '\u09e4', 'ওৎ', 'গথ', 'খঢ', 'ঝ’', ' ং', 'ঔ', 'ডড', 'গঘ'], None)
X = np.zeros((number_of_sample, max_number_of_words, word_vector_size), dtype=K.floatx())
for i in temp.index:
  X[i,:,:] = loaded_model.wv[temp.loc[i,:]]

In [0]:
bengali_news = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/40k_bangla_newspaper_article.p')
bengali_news_dataframe = pd.DataFrame(bengali_news)
y = bengali_news_dataframe['category']

In [0]:
from sklearn import preprocessing
import keras
import numpy as np
le = preprocessing.LabelEncoder()
le.fit(y)
enc = le.transform(y)
y = keras.utils.to_categorical(enc)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle = True, test_size=0.125)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, shuffle = True, test_size=0.20)

In [7]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard


model = Sequential()

model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same',
                 input_shape=(max_number_of_words, word_vector_size)))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=3))

model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.3)))

model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))

model.add(Dense(13, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 50, 32)            3104      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 32)            3104      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 50, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 32)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               591872    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
__________

In [9]:
history = model.fit(X_train, y_train, batch_size= 500, shuffle=True, epochs= 100, validation_data=(X_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 35000 samples, validate on 4000 samples
Epoch 1/100
35000/35000 [==============================] - 10s 298us/step - loss: 2.2375 - acc: 0.2447 - val_loss: 1.9061 - val_acc: 0.3065
Epoch 2/100
35000/35000 [==============================] - 5s 148us/step - loss: 1.9556 - acc: 0.3294 - val_loss: 1.7554 - val_acc: 0.4763
Epoch 3/100
35000/35000 [==============================] - 5s 155us/step - loss: 1.7763 - acc: 0.4440 - val_loss: 1.5907 - val_acc: 0.4902
Epoch 4/100
35000/35000 [==============================] - 5s 151us/step - loss: 1.6414 - acc: 0.4709 - val_loss: 1.4519 - val_acc: 0.4968
Epoch 5/100
35000/35000 [==============================] - 5s 152us/step - loss: 1.5191 - acc: 0.5173 - val_loss: 1.3308 - val_acc: 0.5667
Epoch 6/100
35000/35000 [==============================] - 5s 151us/step - loss: 1.4074 - acc: 0.5585 - val_loss: 1.2216 - val_acc: 0.6370
Epoch 7/100
35000/35000 [==============================] - 5s 151us/

In [0]:
predicts = model.predict(X_test)
import numpy as np
def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [11]:
from sklearn import metrics

print(metrics.accuracy_score(y_test, y_preds))

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

0.815
[[  0   1   0   0   0   0   0   0   0   7   0   0]
 [  0 265   0   5   1   3   9   1   0  22   1   1]
 [  0   3   0   0   0   2   0   0   0   2   0   0]
 [  0  11   0  90   1   0   0   0   0  12   0   3]
 [  0   9   0   0  13   0   0   1   0   3   0   3]
 [  0   3   0   0   0  52   1   0   0   1   0   2]
 [  0   3   0   1   0   3  22   0   0   7   1   1]
 [  0   0   0   0   0   1   0  22   0   3   1   2]
 [  0   1   0   0   0   0   1   0   0   2   0   1]
 [  0  18   0   5   0   1   3   2   0 236   2   3]
 [  0   1   0   0   0   1   1   0   0   0  71   0]
 [  0   1   0   6   0   0   1   0   0   4   0  44]]
                    precision    recall  f1-score   support

art-and-literature       0.00      0.00      0.00         8
        bangladesh       0.84      0.86      0.85       308
       durporobash       0.00      0.00      0.00         7
           economy       0.84      0.77      0.80       117
         education       0.87      0.45      0.59        29
     entertainment  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
accuracy, val_accuracy = np.array(history.history["acc"]), np.array(history.history["val_acc"])
accuracy, val_accuracy = accuracy.reshape(100,1), val_accuracy.reshape(100,1)
accuracies = np.concatenate((accuracy,val_accuracy),axis=1)
np.savetxt('/content/gdrive/My Drive/Projects/Bengali Text Classification/temp.csv',accuracies,delimiter=",")